### 1. Load Packages

In [ ]:
import openai
import pandas as pd
import re
import json

### 2. Load Tokens

In [ ]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [ ]:
df1 = pd.read_csv('processed_60900_61899.csv')
df2 = pd.read_csv('processed_177000_177999.csv')

In [ ]:
df1

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,60900,60900,6555,we already have congestion pricing for taxis &...,2021-07-09 00:02:34+00:00,2021-07-09,2021-27
1,60901,60901,6556,an the best cannot be that much i am sorry htt...,2021-07-08 23:44:03+00:00,2021-07-08,2021-27
2,60902,60902,6557,handicap uber's is getting out of hand,2021-07-08 23:40:10+00:00,2021-07-08,2021-27
3,60903,60903,6558,bitchh i would calmly get in a cab laugh ill b...,2021-07-08 23:16:21+00:00,2021-07-08,2021-27
4,60904,60904,6559,take a cab. you are getting hepatitis walking ...,2021-07-08 23:15:04+00:00,2021-07-08,2021-27
...,...,...,...,...,...,...,...
995,61895,61895,7550,so...i confess ...i walked to target &amp; bou...,2021-06-17 23:27:09+00:00,2021-06-17,2021-24
996,61896,61896,7551,"uber, ferry, plane, train today",2021-06-17 23:00:01+00:00,2021-06-17,2021-24
997,61897,61897,7552,"hi, i order some food about a hour ago and i h...",2021-06-17 22:08:29+00:00,2021-06-17,2021-24
998,61898,61898,7553,i was there. it was an uber. none of the other...,2021-06-17 21:54:38+00:00,2021-06-17,2021-24


In [ ]:
df2

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,177000,177000,7102,please kick japan out of my country they are s...,2021-11-26 13:17:13+00:00,2021-11-26,2021-47
1,177001,177001,7103,once i learned about correctional exercises an...,2021-11-26 12:41:27+00:00,2021-11-26,2021-47
2,177002,177002,7104,empty train cars &gt;&gt;,2021-11-26 12:22:52+00:00,2021-11-26,2021-47
3,177003,177003,7105,that emerald parking lot is about 1 mile walk ...,2021-11-26 12:14:05+00:00,2021-11-26,2021-47
4,177004,177004,7106,"that is going to take time, i need car by end ...",2021-11-26 11:48:54+00:00,2021-11-26,2021-47
...,...,...,...,...,...,...,...
995,177995,177995,8097,how about getting all trash off the sidewalk? ...,2021-06-03 02:03:54+00:00,2021-06-03,2021-22
996,177996,177996,8098,get trash off the sidewalk. trash corals in th...,2021-06-03 02:00:17+00:00,2021-06-03,2021-22
997,177997,177997,8099,people of philadelphia find trae young's hotel...,2021-06-03 01:55:00+00:00,2021-06-03,2021-22
998,177998,177998,8100,ring rain coming back for more and do not you ...,2021-06-03 01:40:50+00:00,2021-06-03,2021-22


### 4.Extract Travel Mode

In [ ]:
def generate_travel_mode(tweet):
    tweet = tweet.lower()  # Ensure the tweet is in lowercase
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an AI trained to extract travel experiences from tweets. Use these demonstrators for guidance:\n"
                "- Example 1: 'The best driver t664893c blocked the bike lane near 341 vanderbilt ave on june 30...' Expected: {'travel_mode': 'car', 'sentiment': 'negative', 'reasons': 'driver blocks the bike lane'}\n"
                "- Example 2: 'Hi. Someone ordered food through you and it was delivered to my door by mistake...' Expected: {'travel_mode': 'na', 'sentiment': 'na', 'reasons': 'na'}\n"
                "- Example 3: '...he is driving so slow...it is a 12 min driving he is making it a 20 min drive.' Expected: {'travel_mode': 'taxi', 'sentiment': 'negative', 'reasons': 'long time to drive'}\n"
                "Avoid assumptions not supported by the tweet's content and do not default to 'na' unless absolutely necessary. Focus on consistency and evidence-based analysis."
            },
            {
                "role": "user",
                "content": f"Analyze the tweet: '{tweet}'. Focus on:\n\n"
                   "1. **Mode of Transportation:** Identify the primary mode (choices: 'car', 'subway', 'bus', 'bike', 'uber', 'lyft', 'taxi', 'na').\n"
                   "2. **Sentiment Analysis:** Determine the sentiment ('positive', 'negative', 'neutral').\n"
                   "3. **Reasoning Behind Sentiment:** Extract reasons influencing the sentiment or use 'na'.\n\n"
                   "Provide a structured summary in JSON format."
            }
        ],
        temperature=0.7,
        max_tokens=250
    )

    if response.choices:
        content = response.choices[0].message.content.lower()  # Ensure output is lowercase
        try:
            result = eval(content)  # Safer parsing might be needed
        except SyntaxError:
            result = {'travel_mode': 'na', 'sentiment': 'na', 'reasons': 'na'}
    else:
        result = {'travel_mode': 'na', 'sentiment': 'na', 'reasons': 'na'}

    # Validate closed form answers
    valid_modes = ['car', 'subway', 'bus', 'bike', 'uber', 'lyft', 'taxi', 'na']
    valid_sentiments = ['positive', 'negative', 'neutral']
    result['travel_mode'] = result['travel_mode'] if result['travel_mode'] in valid_modes else 'na'
    result['sentiment'] = result['sentiment'] if result['sentiment'] in valid_sentiments else 'na'

    # Skip processing if travel mode is 'na'
    if result['travel_mode'] == 'na':
        result['sentiment'] = ''
        result['reasons'] = ''

    return result

In [ ]:
def process_tweet_and_generate_output(df):
    processed_data = []

    for index, row in df.iterrows():
        # Directly get the dictionary output from generate_travel_mode
        analysis_result = generate_travel_mode(row['processed_txt'])

        # Extract and store the relevant information along with the GLOBAL_ID and original tweet
        processed_row = {
            "GLOBAL_ID": row['GLOBAL_ID'],
            "Tweet": row['processed_txt'],
            "Travel Mode": analysis_result.get('travel_mode', 'na'),  # Default to 'na' if missing
            "Sentiment": analysis_result.get('sentiment', 'na'),  # Default to 'na' if missing
            "Reasons": analysis_result.get('reasons', 'na')  # Default to 'na' if missing or empty
        }

        # Append the processed row to the list of processed data
        processed_data.append(processed_row)

    # Convert the processed data into a DataFrame
    return pd.DataFrame(processed_data)

### 5.Outcome

**(1) df1: processed_60900_61899.csv**

In [ ]:
# test
process_tweet_and_generate_output(df1.iloc[0:2])

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing aims to encourage the use o...
1,60901,an the best cannot be that much i am sorry htt...,na,,


In [ ]:
processed_df1 = process_tweet_and_generate_output(df1)
processed_df1.to_csv('Demonstrators_df1.csv', index=False)

In [ ]:
processed_df1

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,car,positive,congestion pricing is intended to encourage th...
1,60901,an the best cannot be that much i am sorry htt...,na,,
2,60902,handicap uber's is getting out of hand,uber,negative,increase in number of handicap ubers causing i...
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,positive,excited about getting into a cab
4,60904,take a cab. you are getting hepatitis walking ...,taxi,negative,"unsafe environment for walking, implying the n..."
...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,uber,positive,"excited about buying a bigger tv for the team,..."
996,61896,"uber, ferry, plane, train today",na,,
997,61897,"hi, i order some food about a hour ago and i h...",na,,
998,61898,i was there. it was an uber. none of the other...,uber,neutral,na


In [ ]:
processed_df1['Travel Mode'].value_counts().get('na', 0)

396

**(2) df2: processed_177000_177999.csv**

In [ ]:
processed_df2 = process_tweet_and_generate_output(df2)
processed_df2.to_csv('Demonstrators_df2.csv', index=False)

In [ ]:
processed_df2

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,177000,please kick japan out of my country they are s...,na,,
1,177001,once i learned about correctional exercises an...,na,,
2,177002,empty train cars &gt;&gt;,subway,positive,empty train cars
3,177003,that emerald parking lot is about 1 mile walk ...,na,,
4,177004,"that is going to take time, i need car by end ...",car,neutral,"expresses need for a car by end of the year, i..."
...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,car,negative,presence of trash on sidewalk and street causi...
996,177996,get trash off the sidewalk. trash corals in th...,car,negative,"congestion caused by cars, lack of space on si..."
997,177997,people of philadelphia find trae young's hotel...,car,negative,disruptive behavior of honking car horns all n...
998,177998,ring rain coming back for more and do not you ...,car,positive,expresses joy and appreciation for the time sp...


In [ ]:
processed_df2['Travel Mode'].value_counts().get('na', 0)

131